Possible questions to check?

Since we have a lot of features due to different types of facilities located very close to each other, the first test we want to do is to determine if there are strong correlations between pairs of independent variables or between an independent and the dependent variable. 

To perform this test, we calculate the Variation Inflation Factor (VIF). The VIF is a measure of colinearity among predictor variables within a multiple regression. It is calculated by taking the the ratio of the variance of all a given model's betas divide by the variane of a single beta if it were fit alone.


We then remove the features that are colinear by taking the average of distances between different facilities that are colinear. 

Next, we plot a heat map between the different predictor variables, and the target variable. 


Followng that, we identify the variables that are particularly significant by doing a significance test on the correlation coefficient. 



4) What are the most appropriate tests to use to analyze these relationships?



In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

import seaborn as sns

from mpl_toolkits.axes_grid1 import make_axes_locatable

import math

from scipy.stats import pearsonr

import time


from scipy.stats import boxcox



from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


from matplotlib import cm

import matplotlib.lines as mlines

sns.set(style = 'whitegrid')
sns.set_palette('bright')
%matplotlib inline

<b> <font size =5> Variation Inflation Factor Scores </b> </font>

In [104]:
## Reading input data
BC = pd.read_csv("Data/BC_input.csv")
NO2 = pd.read_csv("Data/NO2_input.csv")

In [105]:
#Convert columns to non-numeric 
BC = BC._get_numeric_data() #drop non-numeric cols
NO2 = NO2._get_numeric_data() #drop non-numeric cols

In [106]:
# Drop first column
BC.drop(BC.columns[0], axis=1, inplace=True)
NO2.drop(NO2.columns[0], axis=1, inplace=True)

In [110]:
BC.rename(columns = {'BC Value': 'BC_Value'}, inplace = True)

<b> <font size = 4> Step 1: Set up a multiple regression model </b> </font>

In [112]:
features = []
for i in range(1, 95):
    features.append("Q" + "(" + "'" + (str(BC.columns[i])) + "'" + ")")

features = ' + '.join(features)
    
# get y and X dataframes based on this regression:
y, X = dmatrices('BC_Value ~' + features, BC, return_type='dataframe')    


<b> <font size = 4> Step 2: Calculate VIFs  </b> </font>

In [116]:
# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

<b> <font size = 4> Step 3: Inspect VIFs  </b> </font>

In [120]:
vif_df = vif.round(1)

In [124]:
vif_df.tail(50)

,VIF Factor,features
45,1.433351e+06,Q('17244511-Retail-Res-high_dist')
46,5.487402e+05,Q('17250611-Retail-Res-high_dist')
47,8.034968e+12,Q('18128911-Retail-Res-high_dist')
48,3.160421e+13,Q('18134411-Misc-high_dist')
49,4.748697e+05,Q('18135311-Transportation-high_dist')
50,7.078805e+09,Q('18135811-Retail-Res-high_dist')
51,8.341390e+04,Q('18492711-Retail-Res-high_dist')
52,3.280971e+05,Q('18697111-Retail-Res-high_dist')
53,8.620839e+09,Q('18776211-Manufacturing-high_dist')
54,1.371284e+09,Q('18778911-Retail-Res-high_dist')


<b> <font size =5> Plot Heat Map of Correlation Between Features</b> </font>

In [ ]:
BC.drop(columns = ['Unnamed: 0'], inplace=True)
NO2.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
### Plot a heat map - Black Carbon
corr_BC = BC.corr()
arr_corr_BC= corr_BC.as_matrix()

In [ ]:
### Plot a heat map - Nitrogen Dioxide
corr_NO2 = NO2.corr()
arr_corr_NO2= corr_NO2.as_matrix()

In [ ]:
print(plt.get_backend())

# close any existing plots
plt.close("all")

# mask out the top triangle
arr_corr_BC[np.triu_indices_from(arr_corr_BC)] = np.nan

fig, ax = plt.subplots(figsize=(50, 50))

hm = sns.heatmap(arr_corr_BC, cbar=True, vmin = -1, vmax = 1, center = 0,
                 fmt='.2f', annot_kws={'size': 8}, annot=True, 
                 square=False, cmap = 'coolwarm')
#cmap=plt.cm.Blues

ticks = np.arange(corr_BC.shape[0]) + 0.5
ax.set_xticks(ticks)
ax.set_xticklabels(corr_BC.columns, rotation=90, fontsize=20)
ax.set_yticks(ticks)
ax.set_yticklabels(corr_BC.index, rotation=360, fontsize=20)

ax.set_title('Correlation Matrix - Black Carbon', fontsize  = 30)
plt.tight_layout()
#plt.savefig("corr_matrix_incl_anno_double.png", dpi=300)

In [ ]:
print(plt.get_backend())

# close any existing plots
plt.close("all")

# mask out the top triangle
arr_corr_NO2[np.triu_indices_from(arr_corr_NO2)] = np.nan

fig, ax = plt.subplots(figsize=(50, 50))

hm = sns.heatmap(arr_corr_NO2, cbar=True, vmin = -1, vmax = 1, center = 0,
                 fmt='.2f', annot_kws={'size': 8}, annot=True, 
                 square=False, cmap = 'coolwarm')
#cmap=plt.cm.Blues

ticks = np.arange(corr_NO2.shape[0]) + 0.5
ax.set_xticks(ticks)
ax.set_xticklabels(corr_NO2.columns, rotation=90, fontsize=20)
ax.set_yticks(ticks)
ax.set_yticklabels(corr_NO2.index, rotation=360, fontsize=20)

ax.set_title('Correlation Matrix - Nitrogen Oxide', fontsize  = 30)
plt.tight_layout()
#plt.savefig("corr_matrix_incl_anno_double.png", dpi=300)

### Dropping columns that have a correlation above 0.9

columns = np.full((corr_BC.shape[0],), True, dtype=bool)
for i in range(corr_BC.shape[0]):
    for j in range(i+1, corr_BC.shape[0]):
        if corr_BC.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False

selected_columns = BC.columns[columns]
#data = data[selected_columns]

<b> T-test to determine whether is a significant difference between the means of two groups.</b>

1) Is there a significant difference in BC concentration for distance to highways > 4 km and less than 4 km?
1) Is there a significant difference in NO2 concentration for distance to highways > 4 km and less than 4 km?

<b> <font size = 5> Correlation coefficient and Testing the Significance of the Correlation Coefficient </b> </font>

We perform a hypothesis test of the “significance of the correlation coefficient” to decide whether the linear relationship in the data is strong enough to use to model the relationship. Since we have data for the entire population, we can use the population correlation coefficient. 

Null Hypothesis: H$_{0}$: ρ = 0

Alternate Hypothesis: H$_1$: ρ ≠ 0


ρ = population correlation coefficient

Null Hypothesis H$_0$: The population correlation coefficient <b>is not</b> significantly different from zero. There <b>is not</b> a significant linear relationship(correlation) between x and y in the population.

Alternate Hypothesis H$_1$: The population correlation coefficient is significantly different from zero. There <b>is a significant linear relationship</b> (correlation) between x and y in the population.


In [ ]:
pearsonr(BC['BC Value'],BC['Precip'])

In [ ]:
import pingouin as pg
BC_pearsonr = pg.pairwise_corr(BC, method='pearson')

In [ ]:
BC_pearsonr.head()

In [ ]:
#### Estimating the Correlation Coefficient (r), 95% Confidence Interval for r,  R2 and p-value 

In [ ]:
BC_pearsonr_BC = BC_pearsonr[BC_pearsonr.X == 'BC Value']

In [ ]:
BC_pearsonr_BC.head(100)

In [ ]:
## Determine the p-value that is above the level of signifiance alpha of 0.05

In [ ]:
BC_pearsonr[BC_pearsonr['p-unc'] >= 0.005 ]

The above cell indicates that only '10466511-FoodPlant-high_dist' has a p-value that is larger than the significance level α, thus we fail to reject the null hypothesis. Thus, we conclude that there is not enough evidence at the  α level to conclude that there is a linear relationship in the population between the BC value and response '10466511-FoodPlant-high_dist'.

In [ ]:
BC_pearsonr.describe()

In [ ]:
Potential questions: 
    
    1) What is the probability of observing a concentration that is higher than the ambient standards for Black Carbon and NO2? 

In [ ]:
 - Linear regression for stats
    - VIF scores - gives a score between all feeatures/predictor variables
    - Then do a linear regression 
    - Combine two facilities that are close to each other with similar VIF scores and take average distance
    - 
    

In [ ]:
### References
### https://etav.github.io/python/vif_factor_python.html